In [1]:
#pip install mediapipe opencv-python

In [2]:
import cv2
import mediapipe as mp

# Initialize Mediapipe Face Mesh for face landmarks detection
mp_face_mesh = mp.solutions.face_mesh
face_mesh = mp_face_mesh.FaceMesh(static_image_mode=False, max_num_faces=1, min_detection_confidence=0.5)
mp_drawing = mp.solutions.drawing_utils

In [17]:
# Load the virtual glasses image with transparent background (PNG format)
glasses = cv2.imread(r'E:\jupyter_notebook\Pictures\tshirt1.png', cv2.IMREAD_UNCHANGED)
glasses

array([[[255, 255, 255,   0],
        [255, 255, 255,   0],
        [255, 255, 255,   0],
        ...,
        [255, 255, 255,   0],
        [255, 255, 255,   0],
        [255, 255, 255,   0]],

       [[255, 255, 255,   0],
        [255, 255, 255,   0],
        [255, 255, 255,   0],
        ...,
        [255, 255, 255,   0],
        [255, 255, 255,   0],
        [255, 255, 255,   0]],

       [[255, 255, 255,   0],
        [255, 255, 255,   0],
        [255, 255, 255,   0],
        ...,
        [255, 255, 255,   0],
        [255, 255, 255,   0],
        [255, 255, 255,   0]],

       ...,

       [[255, 255, 255,   0],
        [255, 255, 255,   0],
        [255, 255, 255,   0],
        ...,
        [255, 255, 255,   0],
        [255, 255, 255,   0],
        [255, 255, 255,   0]],

       [[255, 255, 255,   0],
        [255, 255, 255,   0],
        [255, 255, 255,   0],
        ...,
        [255, 255, 255,   0],
        [255, 255, 255,   0],
        [255, 255, 255,   0]],

       [[255

In [18]:
cv2.imshow('glass',glasses)

# VIRTUAL TRY ON MODEL using MEDIAPIPE(pose estimation). This works perfectly fine even if your face goes out of the screen.

In [19]:
# Check if the glasses image was loaded correctly
if glasses is None:
    raise FileNotFoundError("Glasses image file not found. Please check the file path.")

# Function to overlay the glasses on the face
def overlay_glasses(frame, face_landmarks, glasses):
    # Get landmarks for the eyes
    left_eye = face_landmarks.landmark[33]  # Left eye landmark
    right_eye = face_landmarks.landmark[263]  # Right eye landmark

    # Convert normalized coordinates to pixel values
    h, w, _ = frame.shape
    left_eye = (int(left_eye.x * w), int(left_eye.y * h))
    right_eye = (int(right_eye.x * w), int(right_eye.y * h))

    # Calculate the width of the glasses based on eye distance
    glasses_width = int(((right_eye[0] - left_eye[0]) ** 2 + (right_eye[1] - left_eye[1]) ** 2) ** 0.5 * 2)

    # Resize the glasses to fit the face
    glasses_resized = cv2.resize(glasses, (glasses_width, int(glasses.shape[0] * (glasses_width / glasses.shape[1]))))

    # Calculate the position to place the glasses
    top_left = (left_eye[0] - glasses_resized.shape[1] // 4, left_eye[1] - glasses_resized.shape[0] // 2)

    # Ensure the overlay does not exceed the frame boundaries
    for i in range(glasses_resized.shape[0]):
        for j in range(glasses_resized.shape[1]):
            # Calculate the pixel position in the frame
            y_pos = top_left[1] + i
            x_pos = top_left[0] + j

            # Check if the position is within the frame boundaries
            if 0 <= x_pos < w and 0 <= y_pos < h and glasses_resized[i, j, 3] > 0:
                frame[y_pos, x_pos] = glasses_resized[i, j, :3]

    return frame

# Start video capture
cap = cv2.VideoCapture(0)

while cap.isOpened():
    success, frame = cap.read()
    if not success:
        break

    # Convert the frame to RGB for Mediapipe processing
    rgb_frame = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)

    # Get face landmarks
    result = face_mesh.process(rgb_frame)

    # If landmarks are detected, overlay the glasses
    if result.multi_face_landmarks:
        for face_landmarks in result.multi_face_landmarks:
            frame = overlay_glasses(frame, face_landmarks, glasses)

    # Display the result
    cv2.imshow('Virtual Try-On', frame)

    if cv2.waitKey(1) & 0xFF == ord('q'):
        break

# Release resources
cap.release()
cv2.destroyAllWindows()


In [13]:
import cv2
import mediapipe as mp
import numpy as np
from moviepy.editor import VideoFileClip

# Initialize Mediapipe Face Mesh for face landmarks detection
mp_face_mesh = mp.solutions.face_mesh
face_mesh = mp_face_mesh.FaceMesh(static_image_mode=False, max_num_faces=1, min_detection_confidence=0.5)
mp_drawing = mp.solutions.drawing_utils

# Load the virtual glasses image with transparent background (PNG format)
glasses = cv2.imread(r'E:\jupyter_notebook\Pictures\sunglass1.png', cv2.IMREAD_UNCHANGED)

# Check if the glasses image was loaded correctly
if glasses is None:
    raise FileNotFoundError("Glasses image file not found. Please check the file path.")

# Function to overlay the glasses on the face with tilt adjustment
def overlay_glasses(frame, face_landmarks, glasses):
    # Get landmarks for the eyes
    left_eye = face_landmarks.landmark[33]  # Left eye landmark
    right_eye = face_landmarks.landmark[263]  # Right eye landmark

    # Convert normalized coordinates to pixel values
    h, w, _ = frame.shape
    left_eye = (int(left_eye.x * w), int(left_eye.y * h))
    right_eye = (int(right_eye.x * w), int(right_eye.y * h))

    # Calculate the width of the glasses based on eye distance
    eye_distance = np.sqrt((right_eye[0] - left_eye[0]) ** 2 + (right_eye[1] - left_eye[1]) ** 2)
    glasses_width = int(eye_distance * 1.2)  # Adding some extra width for a better fit
    glasses_height = int(glasses.shape[0] * (glasses_width / glasses.shape[1]))

    # Resize the glasses to fit the face
    glasses_resized = cv2.resize(glasses, (glasses_width, glasses_height))

    # Calculate the angle of rotation for the glasses
    dx = right_eye[0] - left_eye[0]
    dy = right_eye[1] - left_eye[1]
    angle = np.degrees(np.arctan2(dy, dx))  # Correct angle calculation

    # Calculate the center point for rotation (between the eyes)
    center = (int((left_eye[0] + right_eye[0]) / 2), int((left_eye[1] + right_eye[1]) / 2))

    # Create the rotation matrix
    rotation_matrix = cv2.getRotationMatrix2D(center, angle, 1.0)

    # Rotate the glasses image
    glasses_rotated = cv2.warpAffine(glasses_resized, rotation_matrix, (w, h), flags=cv2.INTER_LINEAR, borderMode=cv2.BORDER_CONSTANT, borderValue=(0, 0, 0, 0))

    # Calculate the position to place the glasses
    top_left = (int(center[0] - glasses_rotated.shape[1] / 2), int(center[1] - glasses_rotated.shape[0] / 2))

    # Ensure the overlay does not exceed the frame boundaries
    for i in range(glasses_rotated.shape[0]):
        for j in range(glasses_rotated.shape[1]):
            # Calculate the pixel position in the frame
            y_pos = top_left[1] + i
            x_pos = top_left[0] + j

            # Check if the position is within the frame boundaries
            if 0 <= x_pos < w and 0 <= y_pos < h and glasses_rotated[i, j, 3] > 0:
                frame[y_pos, x_pos] = glasses_rotated[i, j, :3]

    return frame

# Initialize video capture from the webcam
cap = cv2.VideoCapture(0)

# Get video properties
width = int(cap.get(cv2.CAP_PROP_FRAME_WIDTH))
height = int(cap.get(cv2.CAP_PROP_FRAME_HEIGHT))
fps = int(cap.get(cv2.CAP_PROP_FPS))

# Initialize video writer to save the output video
fourcc = cv2.VideoWriter_fourcc(*'mp4v')
out = cv2.VideoWriter(r'E:\jupyter_notebook\Pictures\sunglass_output_virtual_try.mp4', fourcc, fps, (width, height))

while cap.isOpened():
    success, frame = cap.read()
    if not success:
        break

    # Convert the frame to RGB for Mediapipe processing
    rgb_frame = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)

    # Get face landmarks
    result = face_mesh.process(rgb_frame)

    # If landmarks are detected, overlay the glasses
    if result.multi_face_landmarks:
        for face_landmarks in result.multi_face_landmarks:
            frame = overlay_glasses(frame, face_landmarks, glasses)

    # Write the processed frame to the output video
    out.write(frame)

    # Display the result in real-time
    cv2.imshow('Virtual Try-On', frame)
    if cv2.waitKey(1) & 0xFF == ord('q'):
        break

# Release resources
cap.release()
out.release()
cv2.destroyAllWindows()

# Add the original audio from the live feed (if available) to the saved video
try:
    # Load the saved video
    processed_video = VideoFileClip(r'E:\jupyter_notebook\Pictures\sunglass_output_virtual_try.mp4')

    # Add the original audio (captured by the webcam) if available
    final_video = processed_video.set_audio(processed_video.audio)
    final_video.write_videofile(r'E:\jupyter_notebook\Pictures\sunglass_output_with_audio.mp4', codec="libx264")
except Exception as e:
    print(f"Audio could not be added to the video: {e}")

Moviepy - Building video E:\jupyter_notebook\Pictures\sunglass_output_with_audio.mp4.
Moviepy - Writing video E:\jupyter_notebook\Pictures\sunglass_output_with_audio.mp4



Moviepy - Done !
Moviepy - video ready E:\jupyter_notebook\Pictures\sunglass_output_with_audio.mp4
